# Factorization Machines scoring

In [ ]:
!apt-get install -y build-essential
!apt-get install -y python3-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-dev is already the newest version (3.10.6-1~22.04).
python3-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!pip3 install manimce

  Using cached manimce-0.1.1.post2-py3-none-any.whl (249 kB)
  Using cached cairocffi-1.6.1-py3-none-any.whl (75 kB)
  Using cached pangocairocffi-0.4.0-py3-none-any.whl
  Using cached pangocffi-0.8.0-py3-none-any.whl
  Using cached progressbar-2.5-py3-none-any.whl
  Using cached pycairo-1.25.1.tar.gz (347 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pydub-0.25.1-py2.py3-none-any.whl (32 kB)
  Using cached rich-6.2.0-py3-none-any.whl (150 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for pycairo (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, an

In [ ]:
!pip install river==0.10.1
!pip install numpy --upgrade

In [ ]:
!pip install typing-extensions --upgrade

  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
!pip3 install Cython

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import functions as f
import river
from river import datasets, stream, compose, facto, optim, reco, metrics, preprocessing, feature_extraction
def parse_list(x:str):
    lst = x.strip('][').split('\', ')
    return list(map(lambda x: x.strip('\''), lst))

## Training model
In this section, we trained MF and FM models using River, a python incremeantal learning package, specialize in working with data stream

In [ ]:
# Read Data
def parse_list(x:str):
    try:
      lst = x.strip('][').split('\', ')
      return list(map(lambda x: x.strip('\''), lst))
    except:
      print(f"error for {x}")
      return []
def readData(random_state = None, frac = 0.9):
    df = pd.read_csv('/content/drive/MyDrive/code/ratings_top_w_features.csv', header=0, names=['user','age', 'item', 'rating', 'book_title', 'book_author', 'year_of_publication', 'language', 'categories','country', 'age_group'])
    df['categories'] = df['categories'].apply(parse_list)
    df['country'] = df['country'].apply(str)
    # r_df = df.head(int(0.8* df.shape[0]))
    r_df = df.sample(frac=frac, random_state=random_state) if random_state else df.head(int(frac* df.shape[0]))
    test = df.drop(r_df.index)
    y = r_df.pop('rating')
    y_test = test.pop('rating')
    return stream.iter_pandas(r_df, y), stream.iter_pandas(test, y_test)


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/code/ratings_top_w_features.csv', header=0, names=['user','age', 'item', 'rating', 'book_title', 'book_author', 'year_of_publication', 'language', 'categories','country', 'age_group'],dtype={'country': str, 'categories':str})

In [ ]:
# Calculate weighted
# df = pd.read_csv('data/ratings_top_w_features.csv', header=0, names=['user','age', 'item', 'rating', 'book_title', 'book_author', 'year_of_publication', 'language', 'categories','country', 'age_group'],dtype={'country': str, 'categories':str})
df['categories'] = df['categories'].apply(parse_list)
category_counts = df.explode('categories').groupby('categories')['categories'].count()
weights = 1/category_counts
total_weights = weights.sum()
# min_weight, max_weight = weights.max(), weights.min()
# normalized_weights = (weights - min_weight)/ (max_weight - min_weight)
normalized_weights = weights / total_weights

languages_counts = df.groupby('language')['language'].count()
languages_counts
l_weights = 1 /languages_counts
total_l_weights = l_weights.sum()
normalized_l_weights = l_weights/total_l_weights
category_counts.sort_values(ascending=False), languages_counts.sort_values(ascending=False)

(categories
 Fiction                        135429
 Juvenile Fiction                15062
 Biography & Autobiography        9437
 Humor                            3934
 History                          3354
 Religion                         2981
 Body, Mind & Spirit              2169
 Juvenile Nonfiction              2071
 Social Science                   2059
 Business & Economics             1902
 Family & Relationships           1811
 Self-Help                        1742
 Health & Fitness                 1597
 Cooking                          1377
 Travel                           1230
 Poetry                           1069
 True Crime                       1007
 Psychology                       1002
 Science                           974
 Computers                         945
 Literary Criticism                933
 Drama                             896
 Political Science                 862
 Nature                            821
 Comics & Graphic Novels           816
 Philosophy  

In [ ]:
df.describe()
df['age_group'].unique()

array(['30-40', '20-30', '40-55', '55', '65+', '0-20'], dtype=object)

In [ ]:
df.head()

,user,age,item,rating,book_title,book_author,year_of_publication,language,categories,country,age_group
0,11676,34.7439,0399135782,9,The Kitchen God's Wife,Amy Tan,1991.0,en,[Fiction],NaN,30-40
1,29526,26.0000,0399135782,9,The Kitchen God's Wife,Amy Tan,1991.0,en,[Fiction],usa,20-30
2,46398,37.0000,0399135782,9,The Kitchen God's Wife,Amy Tan,1991.0,en,[Fiction],usa,30-40
3,49635,34.7439,0399135782,9,The Kitchen God's Wife,Amy Tan,1991.0,en,[Fiction],usa,30-40
4,67958,39.0000,0399135782,10,The Kitchen God's Wife,Amy Tan,1991.0,en,[Fiction],usa,30-40


In [ ]:

RANDOM_STATE = 5

In [ ]:
biased_mf_params = {
    'n_factors': 10,
    'bias_optimizer': optim.SGD(0.025),
    'latent_optimizer': optim.SGD(0.05),
    'weight_initializer': optim.initializers.Zeros(),
    'latent_initializer': optim.initializers.Normal(mu=0., sigma=0.1, seed=73),
    'l2_bias': 0.,
    'l2_latent': 0.
}
def trainBiasedMF():
    print('training MF on seed:', RANDOM_STATE)
    res = []
    model = reco.BiasedMF(**biased_mf_params)
    # model = reco.RandomNormal(seed=42)
    metric = metrics.MAE() + metrics.RMSE()
    X_y, X_y_test = readData(RANDOM_STATE)
    cnt = 0
    for x, y in X_y:
        y_pred = model.predict_one(user=x['user'], item=x['item'])

        metric.update(y_pred=y_pred, y_true=y)
        _ = model.learn_one(user=x['user'], item=x['item'], x=x, y=y)
        cnt+= 1
    print('On training', cnt, metric)
    res.extend(metric.get())
    metric = metrics.MAE() + metrics.RMSE()
    cnt = 0
    for x, y in X_y_test:
        y_pred = model.predict_one(user=x['user'], item=x['item'])

        metric.update(y_pred=y_pred, y_true=y)
        cnt+= 1
    print('On test', cnt, metric)
    res.extend(metric.get())
    return model, res

In [ ]:
ffm_params = {
    'n_factors': 10,
    'weight_optimizer': optim.SGD(0.01),
    'latent_optimizer': optim.SGD(0.02),
    'intercept': 3,
    'latent_initializer': optim.initializers.Normal(mu=0., sigma=0.05, seed=73),
}

def split_categories(x):
    categories = x['categories']
    return {f'categorie_{categorie}': normalized_weights[categorie] for categorie in categories}
def weight_language(x):
    lan = x['language']
    return {f'lang_{lan}' : normalized_l_weights[lan]}
def age_bin(x):
    # array(['30-40', '20-30', '40-55', '55', '65+', '0-20', nan], dtype=object)
    return {f'age_{x["age_group"]}':1}
def trainFFM():

    print('training FM on seed:', RANDOM_STATE)
    res = []
    regressor = compose.Select('user') | compose.FuncTransformer(lambda x: {'user': str(x)})
    regressor += (
        compose.Select('item')
    )
    regressor += (
        compose.Select('book_title') | compose.FuncTransformer(lambda x: x['book_title']) | feature_extraction.TFIDF() | preprocessing.Normalizer()
    )
    regressor += (
        compose.Select('age_group') | compose.FuncTransformer(age_bin)
    )
    regressor += (
        compose.Select('categories') | compose.FuncTransformer(split_categories)
    )
    model = facto.FFMRegressor(**ffm_params)

    regressor |= model


    X_y, X_y_test = readData(RANDOM_STATE)
    metric = metrics.MAE() + metrics.RMSE()
    cnt = 0
    for x, y in X_y:
        y_pred = regressor.predict_one(x=x)

        metric.update(y_pred=y_pred, y_true=y)
        _ = regressor.learn_one(x=x, y=y)
        cnt += 1
        if (cnt % 10000 == 0):
            print('On training', cnt, metric)
    print('On training', cnt, metric)
    res.extend(metric.get())
    metric = metrics.MAE() + metrics.RMSE()
    cnt = 0
    for x, y in X_y_test:
        y_pred = regressor.predict_one(x=x)

        metric.update(y_pred=y_pred, y_true=y)
        cnt += 1
    print('On test', cnt, metric)
    res.extend(metric.get())
    return model, res

In [ ]:
train_res = pd.DataFrame(columns=['MF_train_MAE', 'MF_train_RMSE', 'MF_test_MAE', 'MF_test_RMSE','FM_train_MAE', 'FM_train_RMSE', 'FM_test_MAE', 'FM_test_RMSE'])

for RANDOM_STATE in [5]:
    res = trainBiasedMF()[1]
    res.extend(trainFFM()[1])
    train_res.loc[RANDOM_STATE] = res

train_res


training MF on seed: 5
On training 179182 MAE: 1.323061, RMSE: 1.691906
On test 19909 MAE: 1.276311, RMSE: 1.647594
training FM on seed: 5
On training 10000 MAE: 1.45877, RMSE: 1.830751
On training 20000 MAE: 1.429365, RMSE: 1.797247
On training 30000 MAE: 1.402713, RMSE: 1.767244
On training 40000 MAE: 1.387315, RMSE: 1.753499
On training 50000 MAE: 1.377876, RMSE: 1.744838
On training 60000 MAE: 1.369248, RMSE: 1.736042
On training 70000 MAE: 1.361606, RMSE: 1.728639
On training 80000 MAE: 1.357379, RMSE: 1.725579
On training 90000 MAE: 1.353038, RMSE: 1.721631
On training 100000 MAE: 1.34977, RMSE: 1.718653
On training 110000 MAE: 1.345955, RMSE: 1.715761
On training 120000 MAE: 1.340424, RMSE: 1.710071
On training 130000 MAE: 1.334963, RMSE: 1.705109
On training 140000 MAE: 1.332812, RMSE: 1.703547
On training 150000 MAE: 1.329887, RMSE: 1.700715
On training 160000 MAE: 1.328226, RMSE: 1.699278
On training 170000 MAE: 1.324862, RMSE: 1.695774
On training 179182 MAE: 1.322529, RMSE:

,MF_train_MAE,MF_train_RMSE,MF_test_MAE,MF_test_RMSE,FM_train_MAE,FM_train_RMSE,FM_test_MAE,FM_test_RMSE
5,1.323061,1.691906,1.276311,1.647594,1.322529,1.693756,1.257113,1.642227


In [ ]:
import pickle
model, res= trainBiasedMF()
with open('MF_model.pkl','wb') as f:
    pickle.dump(model,f)
fm_model, res = trainFFM()
with open('FM_model.pkl', 'wb') as f:
    pickle.dump(fm_model, f)

training MF on seed: 5
On training 179182 MAE: 1.322855, RMSE: 1.6918
On test 19909 MAE: 1.275566, RMSE: 1.64713
training FM on seed: 5
On training 10000 MAE: 1.454657, RMSE: 1.828525
On training 20000 MAE: 1.42497, RMSE: 1.794552
On training 30000 MAE: 1.399002, RMSE: 1.764861
On training 40000 MAE: 1.384069, RMSE: 1.751339
On training 50000 MAE: 1.37521, RMSE: 1.742953
On training 60000 MAE: 1.366974, RMSE: 1.73438
On training 70000 MAE: 1.359627, RMSE: 1.727184
On training 80000 MAE: 1.355616, RMSE: 1.72437
On training 90000 MAE: 1.351534, RMSE: 1.720662
On training 100000 MAE: 1.348108, RMSE: 1.717512
On training 110000 MAE: 1.344421, RMSE: 1.714568
On training 120000 MAE: 1.339078, RMSE: 1.70908
On training 130000 MAE: 1.333718, RMSE: 1.704213
On training 140000 MAE: 1.331765, RMSE: 1.70275
On training 150000 MAE: 1.328908, RMSE: 1.699914
On training 160000 MAE: 1.327335, RMSE: 1.698506
On training 170000 MAE: 1.32393, RMSE: 1.694956
On training 179182 MAE: 1.321554, RMSE: 1.69287

## Recommendation for user
In this section, we used the trained model to filtered and present a list of recommendation for a specific user
In this example, if the user with id = 193458

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/code/ratings_top_w_features.csv', header=0, names=['user','age', 'item', 'rating', 'book_title', 'book_author', 'year_of_publication', 'language', 'categories','country', 'age_group'],dtype={'country': str})
df['categories'] = df['categories'].apply(parse_list)

In [ ]:
with open('MF_model.pkl', 'rb') as f:
    model = pickle.load(f)
    user_id = 193458
    rated_books = df[df['user'] == user_id]
    rated_books = rated_books[['item', 'book_title', 'categories', 'rating']].drop_duplicates('item')
    # rated_books
    predicted_ratings = []
    for index, row in rated_books.iterrows():
        pred = model.predict_one(user_id, row['item'])
        predicted_ratings.append(pred)
    rated_books['pred_rating'] = predicted_ratings
    rated_books = rated_books.sort_values(by='pred_rating', ascending=False)
rated_books

,item,book_title,categories,rating,pred_rating
6701,0142001740,The Secret Life of Bees,[Fiction],9,8.566909
95506,0064471101,The Magician's Nephew (rack) (Narnia),[Juvenile Fiction],9,8.484360
6227,0064471047,"The Lion, the Witch, and the Wardrobe (The Chr...",[Juvenile Fiction],9,8.336191
23249,0345361792,A Prayer for Owen Meany,[Fiction],10,8.333904
69872,0671880314,Schindler's List,[Fiction],9,8.281133
17518,1853260002,Pride & Prejudice (Wordsworth Classics),[Fiction],10,8.191299
108447,0064471063,The Horse and His Boy,[Juvenile Fiction],9,8.099486
108422,0064471055,Prince Caspian (rack) : The Return to Narnia (...,[Juvenile Fiction],9,8.082659
108469,006447108X,The Last Battle,[Juvenile Fiction],9,8.081337
138327,014011369X,"And the Band Played on: Politics, People, and ...",[History],9,8.040531


In [ ]:
books = df[['item', 'book_title','categories']].drop_duplicates(subset=['item', 'book_title'])
unrated_books = books[~books['item'].isin(rated_books['item'].tolist())].copy()
print(f'all books: {books.shape[0]}, rated books: {rated_books.shape[0]}, not yet rated books: {unrated_books.shape[0]}')

all books: 63819, rated books: 55, not yet rated books: 63764


First we use Matrix Factorization to filterd through the 9383 not yet rated books, and keep 2000 highest predicted rating

In [ ]:
with open('MF_model.pkl', 'rb') as f:
    model = pickle.load(f)
    user_id = 193458
    predicted_ratings = []
    for index, row in unrated_books.iterrows():
        pred = model.predict_one(user_id, row['item'])
        predicted_ratings.append(pred)
    unrated_books['pred_rating'] = predicted_ratings
    unrated_books = unrated_books.sort_values(by='pred_rating', ascending=False)
filtered_books = unrated_books.head(2000).copy()

Then we run Factorization Machine on the 2000 highest predicted rating to get the last recommendation list

In [ ]:
with open('FM_model.pkl', 'rb') as f:
    model = pickle.load(f)
    regressor = compose.Select('user') | compose.FuncTransformer(lambda x: {'user': str(x)})
    regressor += (
        compose.Select('item')
    )
    regressor += (
        compose.Select('book_title') | compose.FuncTransformer(lambda x: x['book_title']) | feature_extraction.TFIDF() | preprocessing.Normalizer()
    )
    regressor += (
        compose.Select('age_group') | compose.FuncTransformer(age_bin)
    )
    regressor += (
        compose.Select('categories') | compose.FuncTransformer(split_categories)
    )

    regressor |= model
    books = df[['item', 'book_title']].drop_duplicates()
    user_id = 193458
    user = df[df['user'] == user_id]
    user = user[['user','age', 'country', 'age_group']].iloc[0].squeeze()
    user_rated = df[df['user'] == user_id]
    user_rated = user_rated[['item', 'book_title', 'rating', 'categories']]
    predicted_ratings = []
    for index, row in filtered_books.iterrows():
        row = pd.concat([row, user])
        pred = regressor.predict_one(row)
        predicted_ratings.append(pred)
    filtered_books['pred_rating'] = predicted_ratings
    filtered_books = filtered_books.sort_values(by='pred_rating', ascending=False)
filtered_books.drop_duplicates('book_title').head(10)

,item,book_title,categories,pred_rating
35337,0836220889,Calvin and Hobbes,[Humor],9.237989
8139,0439139597,Harry Potter and the Goblet of Fire (Book 4),[Juvenile Fiction],9.133762
32439,0812550706,Ender's Game (Ender Wiggins Saga (Paperback)),[Fiction],9.130075
25261,0345339703,The Fellowship of the Ring (The Lord of the Ri...,[Fiction],9.105110
44252,043936213X,Harry Potter and the Sorcerer's Stone (Book 1),[Juvenile Fiction],9.102964
10707,0156528207,The Little Prince,[Juvenile Fiction],9.060291
45211,0064400557,Charlotte's Web (Trophy Newbery),[Juvenile Fiction],9.025727
145294,1880418568,"Wolves of the Calla (The Dark Tower, Book 5)",[Fiction],8.980854
84512,1853261580,The Little Prince (Wordsworth Collection),[Juvenile Fiction],8.939430
49478,0836218221,The Authoritative Calvin and Hobbes (Calvin an...,[Humor],8.929653


In [ ]:
with open('FM_model.pkl', 'rb') as f:
    model = pickle.load(f)
    regressor = compose.Select('user') | compose.FuncTransformer(lambda x: {'user': str(x)})
    regressor += (
        compose.Select('item')
    )
    regressor += (
        compose.Select('book_title') | compose.FuncTransformer(lambda x: x['book_title']) | feature_extraction.TFIDF() | preprocessing.Normalizer()
    )
    regressor += (
        compose.Select('age_group') | compose.FuncTransformer(age_bin)
    )
    regressor += (
        compose.Select('categories') | compose.FuncTransformer(split_categories)
    )

    regressor |= model
    books = df[['item', 'book_title']].drop_duplicates()
    user_id = 193458
    user = df[df['user'] == user_id]
    user = user[['user','age', 'country', 'age_group']].iloc[0].squeeze()
    print(user)
    user_rated = df[df['user'] == user_id]
    user_rated = user_rated[['item', 'book_title', 'rating', 'categories']]
    predicted_ratings = []
    for index, row in user_rated.iterrows():
        row = pd.concat([row, user])
        pred = regressor.predict_one(row)
        predicted_ratings.append(pred)
    user_rated['pred_rating'] = predicted_ratings
    user_rated = user_rated.sort_values(by='pred_rating', ascending=False)
user_rated

user         193458
age            40.0
country      canada
age_group     30-40
Name: 2500, dtype: object


,item,book_title,rating,categories,pred_rating
6701,0142001740,The Secret Life of Bees,9,[Fiction],8.854962
95506,0064471101,The Magician's Nephew (rack) (Narnia),9,[Juvenile Fiction],8.659556
23249,0345361792,A Prayer for Owen Meany,10,[Fiction],8.646336
17518,1853260002,Pride & Prejudice (Wordsworth Classics),10,[Fiction],8.555082
6227,0064471047,"The Lion, the Witch, and the Wardrobe (The Chr...",9,[Juvenile Fiction],8.525952
11382,0345424719,A Widow for One Year,8,[Fiction],8.368827
11238,1853260622,War and Peace (Wordsworth Classics),5,[Fiction],8.297373
69872,0671880314,Schindler's List,9,[Fiction],8.276948
108447,0064471063,The Horse and His Boy,9,[Juvenile Fiction],8.244340
108422,0064471055,Prince Caspian (rack) : The Return to Narnia (...,9,[Juvenile Fiction],8.199750


## Bonus: Implement MF using FM
FM is a generalization of MF, so in theory, we can use FM to reimplementing MF with a similar result

In [ ]:
fm_params = {
    'n_factors': 10,
    'weight_optimizer': optim.SGD(0.025),
    'latent_optimizer': optim.SGD(0.05),
    'sample_normalization': False,
    'l1_weight': 0.,
    'l2_weight': 0.,
    'l1_latent': 0.,
    'l2_latent': 0.,
    'intercept': 3,
    'intercept_lr': .01,
    'weight_initializer': optim.initializers.Zeros(),
    'latent_initializer': optim.initializers.Normal(mu=0., sigma=0.1, seed=73),
}

regressor = compose.Select('user')  | compose.FuncTransformer(lambda x: {'user': str(x)})
regressor += (compose.Select('item'))
regressor |= facto.FMRegressor(**fm_params)

model = preprocessing.PredClipper(
    regressor=regressor,
    y_min=1,
    y_max=10
)
metric = metrics.MAE() + metrics.RMSE()
X_y, z = readData()
cnt = 0
for x, y in X_y:
    y_pred = model.predict_one(x=x)

    metric.update(y_pred=y_pred, y_true=y)
    _ = model.learn_one(x=x, y=y)
    if cnt % 10000 == 0:
        print(cnt, metric)
    cnt+= 1
print(metric)
metric = metrics.MAE() + metrics.RMSE()
for x, y in z:
    y_pred = model.predict_one(x=x)

    metric.update(y_pred=y_pred, y_true=y)
    # _ = model.learn_one(x=x, y=y)
    if cnt % 10000 == 0:
        print(cnt, metric)
    cnt+= 1
print(metric)


0 MAE: 6.019074, RMSE: 6.019074
10000 MAE: 1.395521, RMSE: 1.770745
20000 MAE: 1.380373, RMSE: 1.751652
30000 MAE: 1.360002, RMSE: 1.728276
40000 MAE: 1.346654, RMSE: 1.708966
50000 MAE: 1.341534, RMSE: 1.701696
60000 MAE: 1.341415, RMSE: 1.703185
70000 MAE: 1.337916, RMSE: 1.698332
80000 MAE: 1.332936, RMSE: 1.692468
90000 MAE: 1.329329, RMSE: 1.688843
100000 MAE: 1.329272, RMSE: 1.690474
110000 MAE: 1.329752, RMSE: 1.691807
120000 MAE: 1.32742, RMSE: 1.688552
130000 MAE: 1.328501, RMSE: 1.689557
140000 MAE: 1.325267, RMSE: 1.686469
150000 MAE: 1.32515, RMSE: 1.687107
160000 MAE: 1.323722, RMSE: 1.685505
170000 MAE: 1.324424, RMSE: 1.687489
MAE: 1.317263, RMSE: 1.681476
180000 MAE: 1.463638, RMSE: 1.83557
190000 MAE: 1.390753, RMSE: 1.727151
MAE: 1.397795, RMSE: 1.744796
